In [1]:
from requests import get
from json import loads
from time import sleep
from typing import List
from borsdata_sdk.models.Instrument import Instrument
from borsdata_sdk.models.Market import Market
import pandas as pd
from http import HTTPStatus
from borsdata_sdk import BorsdataAPI
import borsdata_sdk
import requests
from pandas.io.json import json_normalize

In [2]:
auth_key = 'f4e8e3c7c46c4c34ae238a1822863410'

RATE_LIMIT = 429
borsdata = BorsdataAPI(auth_key)
instruments = borsdata.get_instruments()

In [3]:
my_list = []
for i in range(len(instruments)):
    tmp_list = [instruments[i].insId,instruments[i].name,instruments[i].countryId,instruments[i].sectorId,instruments[i].marketId,instruments[i].instrument]
    my_list.append(tmp_list)

instruments_df=pd.DataFrame(my_list,columns=['insID','Name','countryId','sectorId', 'marketId', 'instrument']) 

# Rename the lists to the real names
# MarketId: 1 = Large Cap, 2 = Mid Cap, 3=Small Cap, 4=First North

swe_stock_main = instruments_df[(instruments_df['countryId'] == 1) & (instruments_df['instrument'] == 0) & (instruments_df['marketId'].isin([1,2,3, 4]))]

In [13]:
price_df = pd.DataFrame(columns=['d', 'insID','h','l','c','o','v'])
price_df = price_df.set_index(['d', 'insID'])

for ins in swe_stock_main['insID'].values:
    URL = "https://apiservice.borsdata.se/v1/instruments/"  + str(ins) + "/stockprices/?authKey=f4e8e3c7c46c4c34ae238a1822863410&maxCount=20"
    response = requests.get(URL)
    tmp_df = json_normalize(response.json(), record_path ='stockPricesList')
    
    tmp_df['d'] = pd.to_datetime(tmp_df['d'])
    tmp_df['insID'] = ins
    tmp_df = tmp_df.set_index(['d', 'insID'])
    #price_df.append(tmp_df)
    price_df = pd.concat([price_df,tmp_df], sort=True) 
    



In [36]:
stock_list = swe_stock_main.set_index(['insID'])
export_df = price_df.join(stock_list, how='inner', on = 'insID').sort_index() 

In [37]:

SAVE = 'YES'


if SAVE == 'YES':
    export_df.to_csv("data/SWE_PRICE_20YEAR.csv")